In [1]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv
from process_data import get_sim_events, get_cached_trains_data
from compute import apply_corrections

2025-06-26 12:22:02.136 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.138 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.138 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.139 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.140 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.141 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:02.141 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
all_trains_data = get_cached_trains_data(None)
sim_events = get_sim_events(5)

sim_events

2025-06-26 12:08:24.475 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:08:24.476 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:08:24.656 
  command:

    streamlit run /Users/henrieloy/ideo-POC/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-26 12:08:24.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:08:24.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:08:24.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:08:25.163 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:08:25.164 Thread 'Th

,SIMULATION_ID,MODIFICATION_TYPE,TRAIN_ID,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_POINT,ARRIVAL_POINT,NB_WAGONS,IS_EMPTY
0,5,added,None,2025-05-12 08:00:00,2025-05-12 10:00:00,AMB,AMB,10,False
1,5,added,None,2025-05-12 08:00:00,2025-05-12 10:00:00,AMB,AMB,10,True
2,5,modified,D_EV_AMB_202529749,2025-05-25 21:37:00,2025-05-25 23:59:00,DOU,AMB,21,True
3,5,modified,D_AMB_EV_202529726,2025-05-21 17:38:00,2025-05-25 20:16:00,AMB,EV,28,True
4,5,deleted,D_AMB_EV_202529715,2025-05-22 19:37:00,2025-05-22 22:25:00,AMB,EV,28,False


In [3]:
def apply_simulation(all_trains_data, location, sim_events):
    """Applique une simulation aux stocks"""

    for _, row in sim_events.iterrows():
        if row["MODIFICATION_TYPE"] == "added":
            new_row = {
                "TRAIN_ID": "SIM_"+row["DEPARTURE_POINT"]+"_"+row["ARRIVAL_POINT"]+"_"+row["DEPARTURE_TIME"].strftime("%Y%m%d"),
                "DEPARTURE_POINT": row["DEPARTURE_POINT"],
                "ARRIVAL_POINT": row["ARRIVAL_POINT"],
                "DEPARTURE_DATE": row["DEPARTURE_TIME"],
                "ARRIVAL_DATE": row["ARRIVAL_TIME"],
                "NB_WAGONS": row["NB_WAGONS"],
                "TYPE": "Vides" if row["IS_EMPTY"] else "Chargés",
            }
            all_trains_data = pd.concat([all_trains_data, pd.DataFrame([new_row])], ignore_index=True)
        elif row["MODIFICATION_TYPE"] == "deleted":
            all_trains_data = all_trains_data[all_trains_data["TRAIN_ID"] != row["TRAIN_ID"]]
        elif row["MODIFICATION_TYPE"] == "modified":
            all_trains_data.loc[all_trains_data["TRAIN_ID"] == row["TRAIN_ID"], ["DEPARTURE_DATE", "ARRIVAL_DATE", "DEPARTURE_POINT", "ARRIVAL_POINT", "NB_WAGONS", "TYPE"]] = [
                row["DEPARTURE_TIME"],
                row["ARRIVAL_TIME"], 
                row["DEPARTURE_POINT"],
                row["ARRIVAL_POINT"],
                row["NB_WAGONS"],
                "Vides" if row["IS_EMPTY"] else "Chargés"
            ]

    all_trains_data = all_trains_data.sort_values(by="DEPARTURE_DATE").reset_index(drop=True)
    if location is not None:
        all_trains_data = all_trains_data[
            (all_trains_data["DEPARTURE_POINT"] == location) | 
            (all_trains_data["ARRIVAL_POINT"] == location)
        ]


    return all_trains_data

apply_simulation(all_trains_data, "WOR", sim_events)

,TRAIN_ID,DEPARTURE_POINT,ARRIVAL_POINT,DEPARTURE_DATE,ARRIVAL_DATE,NB_WAGONS,TYPE
38,D_VO_WOR_202529517,VO-GRA-RIO,WOR,2025-05-14 14:35:00,2025-05-16 08:02:00,28,Chargés
58,D_VO_WOR_202529518,VO-GRA-RIO,WOR,2025-05-15 14:35:00,2025-05-20 09:30:00,28,Chargés
71,D_VO_WOR_202529519,VO-GRA-RIO,WOR,2025-05-16 14:35:00,2025-05-21 08:01:00,28,Chargés
106,D_VO_WOR_202529694,VO-GRA-RIO,WOR,2025-05-19 14:35:00,2025-05-22 08:00:00,28,Chargés
125,D_VO_WOR_202529695,VO-GRA-RIO,WOR,2025-05-20 14:35:00,2025-05-23 08:04:00,28,Chargés
143,D_VO_WOR_202529696,VO-GRA-RIO,WOR,2025-05-21 14:35:00,2025-05-23 22:29:00,28,Chargés
162,D_VO_WOR_202529697,VO-GRA-RIO,WOR,2025-05-22 14:35:00,2025-05-27 10:30:00,28,Chargés
179,D_VO_WOR_202529698,VO-GRA-RIO,WOR,2025-05-23 14:35:00,2025-05-28 08:00:00,28,Chargés


In [2]:
stocks_before_sim = apply_corrections("WOR", simulation=False, sim_events=None)
stocks_before_sim

2025-06-26 12:22:05.954 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:05.955 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:22:06.210 
  command:

    streamlit run /Users/henrieloy/ideo-POC/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-26 12:22:06.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:22:06.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:22:06.211 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:22:06.211 No runtime found, using MemoryCacheStorageManager
2025-06-26 12:22:06.716 Thread 'Thread-4': missing ScriptRunContext! This warning ca

,datetime,location,nombre_wagons
0,2025-05-12 00:00:00,WOR,56
1,2025-05-16 08:02:00,WOR,84
2,2025-05-20 09:30:00,WOR,112
3,2025-05-21 08:01:00,WOR,140
4,2025-05-22 08:00:00,WOR,168
5,2025-05-23 08:04:00,WOR,196
6,2025-05-23 22:29:00,WOR,224
7,2025-05-27 10:30:00,WOR,252
8,2025-05-28 08:00:00,WOR,280


In [5]:
stocks_after_sim = apply_corrections("WOR", simulation=True, sim_events=get_sim_events(5))
stocks_after_sim


2025-06-26 12:23:39.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 12:23:39.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,datetime,location,nombre_wagons
0,2025-05-12 00:00:00,WOR,56
1,2025-05-16 08:02:00,WOR,84
2,2025-05-20 09:30:00,WOR,112
3,2025-05-21 08:01:00,WOR,140
4,2025-05-22 08:00:00,WOR,168
5,2025-05-23 08:04:00,WOR,196
6,2025-05-23 22:29:00,WOR,224
7,2025-05-27 10:30:00,WOR,252
8,2025-05-28 08:00:00,WOR,280
